In [1]:
# Install a Drive FUSE wrapper.# Insta 
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:

# Generate creds for the Drive FUSE library.# Gener 
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force

Enter verification code: ··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/


Files in Drive:
4_6030726325214380780.odt
AP-Workshop-Session2.odt
Back transliteration.ods
cat.jpg
Colab Notebooks
converter.pkl
created.txt
cross language survey.odt
ferdosi.txt
fra.txt
google_analogy2.txt
google_analogy.txt
Homeworks-G2.ods
IMG_20141110_092043~2.jpg
Marks.ods
myself.jpg
NER & Word Clustering.ods
New Doc 2.pdf
Open source Tookits or Frameworks or ....ods
outputs
Paper revision.odt
para2vec.odt
Persian corpus information.ods
PhD interview.pdf
PhD presentation.pdf
portrait.jpg
portrait.png
presentation
resized_cat.jpg
resized_myself.jpg
resized_portrait.jpg
resized_starrynight.jpg
simlex999-translated.xlsx
simlex999-translated.xlsx.ods
simlex-final.ods
SNLP_PRESENTATION.pdf
SNLP_PRESENTATION.pptx
SourceCodes
spam.csv
starrynight.jpg
survey.odt
Theoretical Interview.pdf
transliteration.odt
university position.ods
Untitled document.odt
Untitled document.odt (75ca89bf)
Untitled presentation.pdf
Useful things to know.ods
Word embedding.ods
آموزش ورد۲وک.odt
برچسب سوالات.odt

In [13]:
# -*- coding: utf-8 -*-
#
# Copyright 2018 Amir Hadifar. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import metrics

tf.reset_default_graph()

# download universal-sentence-encoder model from tf-hub module
url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

# read dataset (download it from Kaggle.com)
data = pd.read_csv('drive/spam.csv', encoding='latin-1')

# drop unwanted columns, and rename the column name appropriately.
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1": "label", "v2": "text"})
# change string label to integer label
data['label_num'] = data.label.map({'ham': 0, 'spam': 1})

y = list(data['label_num'])
x = list(data['text'])

# split data into test and train
x_train = np.asarray(x[:5000])
y_train = np.asarray(y[:5000])

x_test = np.asarray(x[5000:])
y_test = np.asarray(y[5000:])


# build lambda layer for tf-hub
def UniversalEmbedding(sen):
    return embed(tf.squeeze(tf.cast(sen, tf.string)))


# build Keras model
input_text = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
embedding = tf.keras.layers.Lambda(UniversalEmbedding, output_shape=(512,))(input_text)
dense = tf.keras.layers.Dense(256, activation='relu')(embedding)
pred = tf.keras.layers.Dense(2, activation='softmax')(dense)
model = tf.keras.models.Model(inputs=[input_text], outputs=pred)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# train model

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    model.fit(x_train, y_train, epochs=3, batch_size=128)

    # prediction
    predicts = model.predict(x_test, batch_size=128)

# confusion matrix
predicts = predicts.argmax(axis=1)
metrics.confusion_matrix(y_test, predicts)
print(metrics.classification_report(y_test, predicts))


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda (Lambda)              (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 131,842
Trainable params: 131,842
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
5000/5000 [==============================] - 9s 2ms/step - loss: 0.3174 - acc: 0.8826
Epoch 2/3
5000/5000 [==============================] - 7s 1ms/step - loss: 0.0